In [31]:
import tweepy


In [38]:
import json

# Open the JSON file in read mode using the "with" statement
with open("../secrets.json", 'r') as json_file:
    # Load the data from the JSON file into a Python dictionary
    secrets = json.load(json_file)

# Access the data as a Python dictionary

In [33]:
from requests_oauthlib import OAuth1Session
import os
import json

# In your terminal please set your environment variables by running the following lines of code.
# export 'CONSUMER_KEY'='<your_consumer_key>'
# export 'CONSUMER_SECRET'='<your_consumer_secret>'

consumer_key = secrets['consumer_key']
 
consumer_secret = secrets['consumer_secret']   
    
# User fields are adjustable, options include:
# created_at, description, entities, id, location, name,
# pinned_tweet_id, profile_image_url, protected,
# public_metrics, url, username, verified, and withheld
fields = "created_at,description"
params = {"user.fields": fields}

# Get request token
request_token_url = "https://api.twitter.com/oauth/request_token"
oauth = OAuth1Session(consumer_key, client_secret=consumer_secret)

fetch_response = oauth.fetch_request_token(request_token_url)
#print(fetch_response)



In [34]:
resource_owner_key = fetch_response.get("oauth_token")
resource_owner_secret = fetch_response.get("oauth_token_secret")
print("Got OAuth token: %s" % resource_owner_key)

# # Get authorization
base_authorization_url = "https://api.twitter.com/oauth/authorize"
authorization_url = oauth.authorization_url(base_authorization_url)
print("Please go here and authorize: %s" % authorization_url)

Got OAuth token: 9YK49gAAAAABnNn5AAABiAsxKFQ
Please go here and authorize: https://api.twitter.com/oauth/authorize?oauth_token=9YK49gAAAAABnNn5AAABiAsxKFQ


In [35]:
verifier = input("Paste the PIN here: ")

In [36]:



# Get the access token
access_token_url = "https://api.twitter.com/oauth/access_token"
oauth = OAuth1Session(
    consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=resource_owner_key,
    resource_owner_secret=resource_owner_secret,
    verifier=verifier,
)
oauth_tokens = oauth.fetch_access_token(access_token_url)

access_token = oauth_tokens["oauth_token"]
access_token_secret = oauth_tokens["oauth_token_secret"]

# Make the request
oauth = OAuth1Session(
    consumer_key,
    client_secret=consumer_secret,
    resource_owner_key=access_token,
    resource_owner_secret=access_token_secret,
)

response = oauth.get("https://api.twitter.com/2/users/me", params=params)

if response.status_code != 200:
    raise Exception(
        "Request returned an error: {} {}".format(response.status_code, response.text)
    )

print("Response code: {}".format(response.status_code))

json_response = response.json()

print(json.dumps(json_response, indent=4, sort_keys=True))

Response code: 200
{
    "data": {
        "created_at": "2013-04-29T18:14:33.000Z",
        "description": "",
        "id": "1390126958",
        "name": "Hamish Lamotte",
        "username": "lamotte_hamish"
    }
}


In [42]:
import datetime

In [52]:
import requests
import os
import json

bearer_token = secrets['bearer_token'] 




def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """

    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def save_response(params, tweets, current_utc_time):
    base_path = "./tweets/" + current_utc_time
    with open(base_path + '_tweets.json', 'w') as f:
        json.dump(tweets, f)

    with open(base_path + '_queryparams.json', 'w') as f:
        json.dump(params, f)

def get_recent_tweets(params):
    current_utc_time = datetime.datetime.utcnow().strftime('%Y-%m-%dT%H-%M-%S')
    search_url = "https://api.twitter.com/2/tweets/search/recent"
    response = requests.get(search_url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    save_response(params, response.json(), current_utc_time)
    return response.json()




Docs for filtering tweets query options: https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query

In [54]:
# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields
query_params = {
    'query': 'StarkNet development -is:reply -is:retweet',#'(from:twitterdev -is:retweet) OR #twitterdev',
    'tweet.fields': 'author_id,created_at',
    'max_results': 100
    }
json_response = get_recent_tweets(query_params)


200


In [55]:
query_params = {
    'query': 'StarkNet new project -is:reply -is:retweet',#'(from:twitterdev -is:retweet) OR #twitterdev',
    'tweet.fields': 'author_id,created_at',
    'max_results': 100
    }
json_response = get_recent_tweets(query_params)

200


In [57]:
query_params = {
    'query': 'StarkNet CAIRO -is:reply -is:retweet lang:en',#'(from:twitterdev -is:retweet) OR #twitterdev',
    'tweet.fields': 'author_id,created_at',
    'max_results': 100
    }
json_response = get_recent_tweets(query_params)

200


In [58]:
# filter only mentions of a person to find developers on starknet
query_params = {
    'query': 'StarkNet develop has:mentions -is:reply -is:retweet lang:en',#'(from:twitterdev -is:retweet) OR #twitterdev',
    'tweet.fields': 'author_id,created_at',
    'max_results': 100
    }
json_response = get_recent_tweets(query_params)

200


In [59]:

# filter only mentions of a person to find developers on starknet also has a link that might be to a project
query_params = {
    'query': 'StarkNet develop has:mentions has:links -is:reply -is:retweet lang:en',#'(from:twitterdev -is:retweet) OR #twitterdev',
    'tweet.fields': 'author_id,created_at',
    'max_results': 100
    }
json_response = get_recent_tweets(query_params)

200
